In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from datetime import datetime

ModuleNotFoundError: No module named 'sklearn'

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
hall=pd.read_csv('data.csv')

In [52]:
df=hall.copy()

In [5]:
df= df[df['place']=='콘서트홀']

In [6]:
df=df.drop_duplicates()

In [7]:
df=df.drop('membership_type_6', axis=1)

In [8]:
df=df.drop('place', axis=1)

In [11]:
df[df.price!=0].play_date.nunique()

680

In [32]:
df[(df.play_date.astype(str).str.contains('202002'))&(df.price!=0)].play_date.unique()

array([20200211, 20200215, 20200206, 20200207, 20200222, 20200228,
       20200209, 20200205, 20200208, 20200227, 20200226, 20200219,
       20200229, 20200216, 20200225], dtype=int64)

In [55]:
df.sort_values(by='performance_code').performance_code.unique()

array([   1,    2,    3, ..., 2936, 2937, 2938], dtype=int64)

In [60]:
df[df.performance_code==4]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn
6181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1337,20201112,1930,1층 4열 14,0,0,기획사판매,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
37668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1348,20201112,1930,1층 5열 9,0,0,초대권,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
44613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1348,20201112,1930,1층 11열 17,0,0,초대권,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
46920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201104,2140,20201112,1930,1층 9열 1,0,0,공연진행석,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
96424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1337,20201112,1930,1층 3열 17,0,0,기획사판매,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1348,20201112,1930,1층 5열 13,0,0,초대권,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
1856128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1913,20201112,1930,1층 7열 5,10000,0,초/중/고/대학생 할인50%,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
1877017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1913,20201112,1930,1층 7열 3,10000,0,초/중/고/대학생 할인50%,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N
1893192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201108,1348,20201112,1930,1층 8열 2,0,0,초대권,4,20201020.0,20201019.0,실내악,리사이틀홀,100,10,N


In [62]:
df.iloc[:10,]

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,20220204,2000,3층 BOX9 10,10000,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211003,1550,20211009,2000,1층 7열 5,0,0,초대권,1120,NaN,20210823.0,독주,리사이틀홀,90,15,N
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220917,1520,20221105,2000,1층 C블록 16열 3,0,0,초대권,2283,20220911.0,20220911.0,독주,IBK챔버홀,80,15,N
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190630,1132,20190731,2000,1층 2열 3,0,0,초대권,2798,NaN,20190701.0,독주,리사이틀홀,90,15,N
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,1615,20220302,1930,1층 B블록12열 7,180000,0,일반,897,20220109.0,20220110.0,독주,콘서트홀,90,0,Y
5,30.0,F,블루,무료,NaN,NaN,NaN,NaN,20181124,1145,20190323,2000,1층 A블록2열 1,144000,2,블루회원 할인20%,528,20181119.0,20181119.0,교향곡,콘서트홀,100,15,Y
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191020,1459,20191109,1700,3층 E블록4열 8,0,0,초대권,44,NaN,20191023.0,교향곡,콘서트홀,100,15,N
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,954,20190723,2000,2층 D블록8열 4,0,0,초대권,827,20190602.0,20190602.0,교향곡,콘서트홀,120,20,N
8,30.0,F,골드,무료,NaN,NaN,NaN,NaN,20220114,1101,20220317,1930,1층 B블록 11열 9,56000,2,골드회원 할인20%,1631,20220114.0,20220115.0,실내악,IBK챔버홀,80,15,Y
9,NaN,F,블루,무료,NaN,NaN,NaN,NaN,20190703,908,20190721,1700,1층 C블록17열 3,75000,0,일반,1528,NaN,20190413.0,클래식,콘서트홀,100,15,N


In [13]:
df

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,running_time,intermission,member_yn
0,50.0,F,골드,무료,그린,NaN,NaN,20220114,1512,20220204,2000,3층 BOX9 10,10000,2,일반,302,20220114.0,20220115.0,교향곡,120,15,Y
4,50.0,M,무료,NaN,NaN,NaN,NaN,20220206,1615,20220302,1930,1층 B블록12열 7,180000,0,일반,897,20220109.0,20220110.0,독주,90,0,Y
5,30.0,F,블루,무료,NaN,NaN,NaN,20181124,1145,20190323,2000,1층 A블록2열 1,144000,2,블루회원 할인20%,528,20181119.0,20181119.0,교향곡,100,15,Y
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191020,1459,20191109,1700,3층 E블록4열 8,0,0,초대권,44,NaN,20191023.0,교향곡,100,15,N
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190613,954,20190723,2000,2층 D블록8열 4,0,0,초대권,827,20190602.0,20190602.0,교향곡,120,20,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920853,60.0,F,무료,그린,NaN,NaN,NaN,20210618,1509,20210704,1500,1층 A블록2열 2,90000,2,일반,887,20210618.0,20210620.0,교향곡,110,20,Y
1920855,50.0,M,무료,NaN,NaN,NaN,NaN,20230522,1729,20230613,1700,3층 A블록6열 4,10000,0,일반,2825,20230505.0,20230507.0,교향곡,120,15,Y
1920856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201009,1652,20201020,1930,1층 D블록16열 12,0,0,초대권,77,NaN,20200920.0,클래식,120,15,N
1920858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200726,1655,20200818,1930,1층 D블록20열 8,0,0,기획사판매,850,20200719.0,20200720.0,합창,110,15,N


### 공연명 코드가 같으면 공연 정보와 대관 정보가 같은가?

In [26]:
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'tran_date', 'tran_time', 'play_date', 'play_st_time', 'seat', 'price',
       'ticket_cancel', 'discount_type', 'performance_code', 'pre_open_date',
       'open_date', 'genre', 'running_time', 'intermission', 'member_yn'],
      dtype='object')

In [ ]:
test=df.loc[:,['performance_code', 'play_date', 'play_st_time', 'pre_open_date', 'open_date', 'genre', 'running_time', 'intermission']]

In [ ]:
test['same']=''

In [ ]:
grouped=test.groupby('performance_code')

In [ ]:
non_same_performance_code=pd.DataFrame(test.groupby('performance_code').apply(lambda group: 'y' if group.loc[:,'genre'].nunique() <= 1 else 'n').reset_index(drop=True))

In [210]:
non_same_performance_code['performance_code']=grouped.groups.keys()

In [211]:
non_same_performance_code=non_same_performance_code[non_same_performance_code[0]=='n']['performance_code']

In [212]:
non_same_performance_code

2         7
245     646
272     863
432    1496
493    1863
509    2046
692    2839
Name: performance_code, dtype: int64

In [343]:
test[test['performance_code']==non_same_performance_code[692]]['genre'].unique()

SyntaxError: invalid syntax (727379064.py, line 1)

### 예매 개수 찾기

In [325]:
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'tran_date', 'tran_time', 'play_date', 'play_st_time', 'seat', 'price',
       'ticket_cancel', 'discount_type', 'performance_code', 'pre_open_date',
       'open_date', 'genre', 'running_time', 'intermission', 'member_yn'],
      dtype='object')

In [326]:
indices2=['discount_type','seat']

In [327]:
df_duplicated=df.loc[:,~df.columns.isin(indices2)][df['price']!=0][df[(df['price']!=0)].loc[:,~df.columns.isin(indices2)].duplicated(keep=False)]

In [328]:
df_duplicated['pay_count']=1

In [329]:
df_dup=df_duplicated.iloc[:,:-1]

In [330]:
df_dup=df_dup.fillna(-1)

In [331]:
df_index=df_dup.index

In [341]:
time=datetime.now()
for index in df_index:
    row=df_dup.loc[index,:]
    if df_duplicated.loc[index, 'pay_count']==1:
        condition=(df_dup == row).all(axis=1)
        df_duplicated.loc[condition,'pay_count']=condition.sum()
datetime.now()-time

datetime.timedelta(seconds=1302, microseconds=301798)

In [340]:
df_index=df_duplicated[df_duplicated['pay_count']==1].index

In [ ]:
'''
def extract_percentage(value):
    match = re.search(r'(\d+)%', value)
    if match:
        return int(match.group(1))
    return None
'''